# This notebook sets some prereqs if running in dockerized environment such as https://mybinder.org

*** Important: This notebook will auto close if unused for 10 minutes. If that occurs reload from the initial url. ***



## Feature key must be set (make sure this is not shared or placed in a public place) and aerospike database server started.

In [ ]:
import os
import subprocess
import pprint
import time
features = """
COPY FEATURE FILE HERE
"""

features_file = open("/etc/aerospike/features.conf", "w")
n = features_file.write(features)
features_file.close()


os.system("/entrypoint.sh")

pp = pprint.PrettyPrinter(indent=4)

pp.pprint(subprocess.run(["asd"], stdout=subprocess.PIPE, stderr=subprocess.STDOUT).stdout.splitlines())

pp.pprint(subprocess.run(["whoami"], stdout=subprocess.PIPE).stdout.splitlines())

time.sleep(5) # may not be long enough for server to finish starting. Check the logs in the next step.

### Wait until aerospike has started correctly by checking the logs

Note: If you need to check the aerospike logs at any time, rerun this.

In [ ]:

pp.pprint(subprocess.run(["cat", "/tmp/aerolog"], stdout=subprocess.PIPE, stderr=subprocess.STDOUT).stdout.splitlines())

## Execute basic python example to ensure database is connected

In [ ]:


# import the module
from __future__ import print_function
import aerospike

# Configure the client
config = {
  'hosts': [ ('127.0.0.1', 3000) ]
}

# Create a client and connect it to the cluster
try:
  client = aerospike.client(config).connect()
except:
  import sys
  print("failed to connect to the cluster with", config['hosts'])
  sys.exit(1)

# Records are addressable via a tuple of (namespace, set, key)
key = ('test', 'demo', 'foo')

try:
  # Write a record
  client.put(key, {
    'name': 'John Doe',
    'age': 32
  })
except Exception as e:
  import sys
  print("error: {0}".format(e), file=sys.stderr)

# Read a record
(key, metadata, record) = client.get(key)
print(record)

# Close the connection to the Aerospike cluster
client.close()